**Анализ базы данных сервиса для чтения книг по подписке**



    Компания  купила крупный сервис для чтения книг по подписке. 
    
    Проанализирована база данных, в которой есть информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Результаты анализа помогут сформулировать ценностное предложение для нового продукта.


In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных

In [4]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Знакомство с данными

In [5]:
query = (
'''
SELECT *
FROM books
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

In [6]:
query = (
'''
SELECT *
FROM authors
LIMIT 5 ;

''')

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

In [7]:
query = (
'''
select *
from publishers
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

In [8]:
query = (
'''
SELECT *
FROM ratings
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

In [9]:
query = (
'''
SELECT *
FROM reviews
LIMIT 5;

''')

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзор

## Cколько книг вышло после 1 января 2000 года

In [10]:
query = (
'''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01'
;

''')

pd.io.sql.read_sql(query, con = engine)

,count
0,821


## Количество обзоров и средняя оценка для каждой книги

In [11]:
query = (
'''
WITH
re AS (SELECT b.book_id AS book_id,
              b.title AS title,
              COUNT (re.review_id) AS count_review
       FROM books AS b
       LEFT JOIN reviews AS re ON b.book_id=re.book_id
       GROUP BY b.book_id),
ra AS (SELECT b.book_id AS book_id,
              AVG (ra.rating) AS avg_rating
       FROM books AS b
       LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
       GROUP BY b.book_id)
SELECT re.book_id,
       re.title,
       re.count_review,
       ra.avg_rating
       
FROM re JOIN ra ON re.book_id=ra.book_id
;

''')

pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_review,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


## Издательство, которое выпустило наибольшее число книг, кроме брошюр. 

Брошюрами считаем книги меньше 50 страниц.

In [12]:
query = (
'''
SELECT p.publisher,
       COUNT (b.book_id) AS count_books

FROM books AS b
JOIN publishers AS p ON b.publisher_id=p.publisher_id
WHERE num_pages >50

GROUP BY p.publisher_id
ORDER BY count_books DESC
LIMIT 1;

''')

pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


## Автор с самой высокой средней оценкой книг,  учитывали только книги с 50 и более оценками

In [16]:
query = (
'''
SELECT a.author_id,
       a.author,
       AVG(ra.rating) 
       
FROM (SELECT book_id,
             COUNT(rating_id)
             
      FROM ratings
      GROUP BY book_id
      HAVING COUNT(rating_id) > 50) AS cr
LEFT JOIN ratings AS ra ON cr.book_id=ra.book_id
LEFT JOIN books AS b ON cr.book_id=b.book_id
LEFT JOIN authors AS a ON b.author_id=a.author_id

GROUP BY a.author_id
ORDER BY AVG(ra.rating) DESC

LIMIT 1;

''')

pd.io.sql.read_sql(query, con = engine)

,author_id,author,avg
0,236,J.K. Rowling/Mary GrandPré,4.287097


## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [14]:
query = (
'''
SELECT COUNT (review_id) / COUNT(DISTINCT username)
FROM reviews
WHERE username IN (SELECT username
                   FROM ratings
                   GROUP BY username
                   HAVING COUNT(rating_id) > 50)



''')

pd.io.sql.read_sql(query, con = engine)

,?column?
0,24
